In [39]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import timm
import torch

# sns.set_theme()

%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

torch.manual_seed(42)

Need a model that takes a sequence of images as input and chooses two among them for the best exposure fusion result

In [52]:
from math import comb

import pandas as pd
import torchvision as tv
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms

from dhdrnet.dataset import CachingDataset
from dhdrnet.free_choice_model import FreeChoiceDHDRNet
from dhdrnet.fc_train import get_loaders

exposure_vals = [-5, -3, -1, 0, 1, 3, 5]

train_file = Path("../../precomputed_data/train_current.csv")
train_image_names = pd.read_csv(train_file, squeeze=True, dtype=str).to_list()

ds = CachingDataset(
    data_dir=Path("../../data"),
    image_names=train_image_names,
    exposure_values=exposure_vals,
    transform=transforms.Compose(
        [transforms.ToPILImage(), transforms.Resize((300, 300)), transforms.ToTensor()]
    ),
)
tdl, vdl = get_loaders(ds, 4, trainval_split=0.8)

train_len=2316
val_len=579


In [51]:
for batch in dl:
    images, ev_classes, scores = batch
    print(f"{images.shape=}")
    print(f"{ev_classes}")
    break

images.shape=torch.Size([3, 7, 3, 300, 300])
tensor([16,  4, 12])


datasets are being processed by the dataloader correctly :)

In [5]:
train_ds = Subset(ds, range(20))
val_ds = Subset(ds, range(20, 40))

In [6]:
num_classes = comb(len(exposure_vals), 2)
model = FreeChoiceDHDRNet(num_classes)
fit(
    model,
    train_ds,
    val_ds,
    batch_size=10,
    num_epochs=10,
    exp_name="blank_ds",
    logdir="../../logs",
)

Train:   0%|          | 0/2 [02:27<?, ?it/s, train_loss=3.1, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [04:50<?, ?it/s, val_loss=3.06, batch_idx=0]

Val: 100%|██████████| 1/1 [04:50<00:00, 290.55s/it, val_loss=3.06, batch_idx=0]

                                                                               
Train:   0%|          | 0/2 [00:03<?, ?it/s, train_loss=2.54, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [00:04<?, ?it/s, val_loss=3.02, batch_idx=0]

Val: 100%|██████████| 1/1 [00:04<00:00,  4.11s/it, val_loss=3.02, batch_idx=0]

                                                                              
Train:   0%|          | 0/2 [00:03<?, ?it/s, train_loss=2.02, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [00:03<?, ?it/s, val_loss=2.88, batch_idx=0]

Val: 100%|██████████| 1/1 [00:03<00:00,  3.98s/it, val_loss=2.88, batch_idx=0]

                         

In [16]:
tf = Path("../../precomputed_data/test_current.csv")
pd.read_csv(tf, squeeze=True)

0      0030_20151008_105249_589
1      5066_20160724_121618_110
2      0037_20160719_125227_729
3      0543_20150918_175650_498
4      5a9e_20150404_131348_909
                 ...           
719    0543_20150918_072911_757
720    0043_20160908_100556_696
721    9bf4_20150824_204210_479
722    33TJ_20150607_020736_953
723    0043_20160923_122514_781
Name: names, Length: 724, dtype: object

In [31]:
parts = (f.stem.split("_")[-1] for f in Path("../../logs/").glob("exp_*"))
max(int(p) for p in parts if p.isdigit())

10

In [38]:
from dhdrnet.util import get_valid_exp_path

get_valid_exp_path(Path("../../logs"))

PosixPath('../../logs/exp_11')